In [1]:
from sqlalchemy import create_engine, exc
import pandas as pd

In [2]:
usuario = 'cursopython_aluno16'
senha = 'Treina2017'
host = 'cursopython.dev.br'
port = 3306
banco = 'cursopython_vendas16'

In [3]:
try:
    engine = create_engine(f"mysql+pymysql://{usuario}:{senha}@{host}:{port}/{banco}")
    with engine.connect() as connori:
        print(f"Conexão com o banco '{banco}' estabelecida com sucesso.")
except exc.SQLAlchemyError as e:
    print(f"Erro ao conectar ao banco '{banco}': {e}")
    exit()

Conexão com o banco 'cursopython_vendas16' estabelecida com sucesso.


In [4]:
sql = f"""
     select cl.id,cl.nome,cl.sexo,cl.profissao,cl.nacionalidade,pa2.pais as nac,cl.email,cl.nascimento,
            cl.cadastro, cl.endereco,cl.bairro,cl.cidade,cl.estado,cl.cep,pa1.pais
     from clientes cl
     inner join paises pa1 on cl.pais = pa1.sigla
     inner join paises pa2 on cl.nacionalidade = pa2.sigla
     """
clientes = pd.read_sql(sql,engine)
clientes = clientes.reset_index()
clientes['provedor'] = clientes['email'].apply(lambda x: x.split('@')[1])
clientes['enderecocompleto'] = clientes.apply(lambda row: 
    f"{row['endereco']}, {row['bairro']}, {row['cidade']} - {row['estado']}, CEP: {row['cep']},{row['pais']}",
    axis=1
)
clientes['sexodescricao'] = clientes['sexo'].map({
    'f': 'Feminino',
    'F': 'Feminino',
    'm': 'Masculino',
    'M': 'Masculino',
    'o': 'Outros',
    'O': 'Outros'
})
clientes.columns=['id', 'id_transacional', 'nome_cliente', 'sexo_codigo', 'profissao', 
       'cod_nacionalidade', 'nacionalidade',
       'email', 'nascimento', 'cadastro', 'endereco', 'bairro', 'cidade',
       'estado', 'cep', 'pais', 'provedor_do_cliente', 'endereco_completo',
       'sexo_descricao']
clientes=clientes[['id', 'id_transacional', 'nome_cliente', 'sexo_codigo','sexo_descricao',
                   'cod_nacionalidade', 'nacionalidade','profissao','email',
                   'provedor_do_cliente','nascimento', 'cadastro','endereco_completo']]
                   
clientes['id'] += 1
try:
    clientes.to_sql('dim_cliente',if_exists='append',index=False,con=engine)
    print("Dimensão cliente carregada com sucesso.")
except exc.SQLAlchemyError as e:
        print(f"Erro ao carregar: {e}")


Dimensão cliente carregada com sucesso.
